# Global Financial Centers Data Mining

In [1]:
import pandas as pd
import numpy as np

import re

import json
import requests
from urllib.parse import quote_plus

import matplotlib.pyplot as plt
import geopandas as gpd
import geoplot as gplt
from geojson import Feature, FeatureCollection, dump
import folium
from shapely.geometry import box, mapping, polygon, multipolygon
from foursquare import generate_square_grid, plot_foursquare_search_grid

In [41]:
# Request list of Global Financial Centers and their Global Financial Centres Index (GFCI) from Wikipedia

wiki_url = "https://en.wikipedia.org/wiki/Global_Financial_Centres_Index"

html_content=requests.get(wiki_url).content

# Find two-column dataframe and drop unneeded columns
df = pd.read_html(html_content)[1].append(pd.read_html(html_content)[2])
df.drop(columns=['Rank', 'Change', 'Change.1'], inplace=True)

# Exclude city names' parenthetical notes
df.Centre = df.Centre.str.replace(' \(BOT\)', "").values
df.Centre = df.Centre.str.replace(' \(CD\)', "").values
df.head()

,Centre,Rating
0,New York City,790
1,London,773
2,Hong Kong,771
3,Singapore,762
4,Shanghai,761


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 0 to 51
Data columns (total 2 columns):
Centre    104 non-null object
Rating    104 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [90]:
# df.to_csv('data/cities_ratings.csv')

df = pd.read_csv('data/cities_ratings.csv', index_col=0)
df.head()

,Centre,Rating
0,New York City,790
1,London,773
2,Hong Kong,771
3,Singapore,762
4,Shanghai,761


In [24]:
non_financial_cities = "https://en.wikipedia.org/wiki/List_of_largest_cities"

non_financial_html=requests.get(non_financial_cities).content
df_non_financial = pd.read_html(non_financial_html)[1]
df_non_financial

City        Country Skyline                 City proper  \
             City        Country Skyline                  Definition   
0           Tokyo          Japan     NaN       Metropolis prefecture   
1         Jakarta      Indonesia     NaN      Special capital region   
2       Chongqing          China     NaN                Municipality   
3          Manila    Philippines     NaN                        City   
4           Delhi          India     NaN  National capital territory   
..            ...            ...     ...                         ...   
73     Düsseldorf        Germany     NaN              Urban district   
74         Madrid          Spain     NaN                Municipality   
75        Toronto         Canada     NaN                        City   
76          Surat          India     NaN                Municipality   
77  San Francisco  United States     NaN    Consolidated city-county   

                              Metropolitan area             Urban area[9]  \
        Population  Area(km2)        Population   Area(km2)    Population   
0   13,515,271[10]  2,191[10]    37,274,000[11]  13,452[11]      38505000   
1   10,154,134[12]    664[12]    33,430,285[13]   7,063[13]      34365000   
2   30,752,000[14]      82403               NaN         NaN       8300000   
3    1,780,148[15]         43    12,877,253[15]         620      29700000   
4   16,753,235[16]       1484    29,000,000[17]   3,483[17]      28125000   
..             ...        ...               ...         ...           ...   
73     642,304[86]        217     1,555,985[48]         NaN       6660000   
74   3,266,126[87]        606     6,641,649[48]         NaN       6345000   
75   2,503,281[88]    630[88]     5,928,040[89]   5,906[89]       6630000   
76   4,466,826[90]    327[90]               NaN         NaN       6385000   
77     883,305[25]        122     4,729,484[91]   6,417[91]       6100000   

              
   Area(km2)  
0   8,223[a]  
1       3367  
2   1,489[b]  
3   1,813[c]  
4       2240  
..       ...  
73  2,655[m]  
74      1360  
75      2300  
76       233  
77  2,865[n]  

[78 rows x 10 columns]

In [25]:
non_financial_cities = [value for value in df_non_financial['City']['City'] if value not in list(df.Centre)] 
non_financial_cities

['Chongqing',
 'Delhi',
 'Lagos',
 'Dhaka',
 'Cairo',
 'Karachi',
 'Kolkata',
 'Tehran',
 'Kinshasa',
 'Baoding',
 'Lahore',
 'Lima',
 'Bangalore',
 'Ho Chi Minh City',
 'Harbin',
 'Wuhan',
 'Shijiazhuang',
 'Bogotá',
 'Suzhou',
 'Linyi',
 'Chennai',
 'Nagoya',
 'Nanyang',
 'Zhengzhou',
 'Hyderabad',
 'Surabaya',
 "Xi'an",
 'Quanzhou',
 'Dongguan',
 'Bandung',
 'Hanoi',
 'Shenyang',
 'Baghdad',
 'Onitsha',
 'Ahmedabad',
 'Luanda',
 'Dallas',
 'Pune',
 'Houston',
 'Düsseldorf',
 'Surat']

In [26]:
# drop Delhi since exists in top centres list as New Delhi
del non_financial_cities[1]
non_financial_cities

['Chongqing',
 'Lagos',
 'Dhaka',
 'Cairo',
 'Karachi',
 'Kolkata',
 'Tehran',
 'Kinshasa',
 'Baoding',
 'Lahore',
 'Lima',
 'Bangalore',
 'Ho Chi Minh City',
 'Harbin',
 'Wuhan',
 'Shijiazhuang',
 'Bogotá',
 'Suzhou',
 'Linyi',
 'Chennai',
 'Nagoya',
 'Nanyang',
 'Zhengzhou',
 'Hyderabad',
 'Surabaya',
 "Xi'an",
 'Quanzhou',
 'Dongguan',
 'Bandung',
 'Hanoi',
 'Shenyang',
 'Baghdad',
 'Onitsha',
 'Ahmedabad',
 'Luanda',
 'Dallas',
 'Pune',
 'Houston',
 'Düsseldorf',
 'Surat']

In [27]:
len(non_financial_cities)

40

In [28]:
# More populous cities hand-selected from to introduce a wide variety of non-financial cities
# They represent a mix of capitals and non-capitals 

africa_balancer = [
    'Ouagadougou',
    'Abidjan',
    'Addis Ababa',
    'Kumasi',
    'Conakry',
    'Nairobi',
    'Antananarivo',
    'Fès',
    'Maputo',
    'Niamey',
    'Dakar',
    'Freetown',
    'Mogadishu',
    'Kampala',
    'Dar es Salaam',
    'Lusaka',
    'Harare'
]

americas_balancer = [
    'Calgary',
    'Montréal',
    'Havana',
    'Santo Domingo',
    'Tegucigalpa',
    'Guadalajara',
    'Juárez',
    'La Laguna',
    'León',
    'Monterrey',
    'Puebla-Tlaxcala',
    'Querétaro',
    'San Luis Potosí-Soledad de Graciano Sánchez',
    'Tijuana',
    'Toluca',
    'Philadelphia',
    'Phoenix',
    'San Antonio',
    'San Diego',
    'Córdoba',
    'Mendoza',
    'Rosario',
    'Santa Cruz',
    'Belém',
    'Belo Horizonte',
    'Campinas',
    'Fortaleza',
    'Guarulhos',
    'Manaus',
    'Salvador',
    'Barranquilla',
    'Cali',
    'Cartagena',
    'Medellín',
    'Esmeraldas',
    'QUITO',
    'Montevideo',
    'Caracas',
    'Maracaibo'
]

asia_balancer = [
    'Kabul Center',
    'Yerevan',
    'Baku',
    'Chittagong',
    'Phnom Penh',
    'Bat Dambang',
    'Tbilisi',
    'Agra',
    'Amritsar',
    'Aurangabad',
    'Bhopal',
    'Dhanbad',
    'Indore',
    'Kalyan Dombivali',
    'Kanpur',
    'Calcutta',
    'Lucknow',
    'Ludhiana',
    'Meerut',
    'Nagpur',
    'Nashik',
    'Patna',
    'Pimpri Chinchwad',
    'Rajkot',
    'Srinagar',
    'Thane',
    'Vadodara',
    'Vasai-Virar City',
    'Batam',
    'Bogor',
    'Makasar',
    'Medan',
    'Surabaya',
    'Tangerang',
    'Ahwaz',
    'Esfahan',
    'Karaj',
    'Mashhad',
    'Qom',
    'Tabriz',
    'Al-Basrah Qadha Center',
    'Al-Mosal Qadha Center',
    'Jerusalem',
    'Fukuoka',
    'Kobe',
    'Kawasaki',
    'Kyoto',
    'Sapporo',
    'Yokohama',
    'Amman',
    'Astana',
    'Ulannbaatar',
    'Mandalay',
    'Yangon',
    'Kathmandu',
    'Caloocan',
    'Davao',
    'Quezon City',
    'Busan',
    'Daegu',
    'Incheon',
    'Ulsan',
    'Jiddah',
    'Makkah',
    'Aleppo',
    'Hama',
    'Homs',
    'Idleb',
    'Chon Buri',
    'Samut Prakan',
    'Adana',
    'Antalya',
    'Balıkesir',
    'Bursa',
    'Diyarbakir',
    'Izmir',
    'Gaziantep',
    'Hatay',
    'Kocaeli',
    'Konya',
    'Manisa',
    'Mersin',
    'Sanliurfa',
    'Tashkent',
    "Sana'a"
]

europe_balancer = [
    'Wien',
    'Minsk',
    'Zagreb',
    'Praha',
    'Lille',
    'Marseille',
    'München',
    'Bucharest',
    'Ekaterinburg',
    'Kazan',
    'Nizhny Novgorod',
    'Novosibirsk',
    'Omsk',
    'Ufa',
    'Belgrade',
    'Barcelona',
    'Kharkiv',
    'Birmingham',
]

oceania_balancer = [
    'Adelaide',
    'Brisbaine',
    'Perth',
    'Auckland'
]

balancers = africa_balancer + americas_balancer + asia_balancer + europe_balancer + oceania_balancer
balancers

['Ouagadougou',
 'Abidjan',
 'Addis Ababa',
 'Kumasi',
 'Conakry',
 'Nairobi',
 'Antananarivo',
 'Fès',
 'Maputo',
 'Niamey',
 'Dakar',
 'Freetown',
 'Mogadishu',
 'Kampala',
 'Dar es Salaam',
 'Lusaka',
 'Harare',
 'Calgary',
 'Montréal',
 'Havana',
 'Santo Domingo',
 'Tegucigalpa',
 'Guadalajara',
 'Juárez',
 'La Laguna',
 'León',
 'Monterrey',
 'Puebla-Tlaxcala',
 'Querétaro',
 'San Luis Potosí-Soledad de Graciano Sánchez',
 'Tijuana',
 'Toluca',
 'Philadelphia',
 'Phoenix',
 'San Antonio',
 'San Diego',
 'Córdoba',
 'Mendoza',
 'Rosario',
 'Santa Cruz',
 'Belém',
 'Belo Horizonte',
 'Campinas',
 'Fortaleza',
 'Guarulhos',
 'Manaus',
 'Salvador',
 'Barranquilla',
 'Cali',
 'Cartagena',
 'Medellín',
 'Esmeraldas',
 'QUITO',
 'Montevideo',
 'Caracas',
 'Maracaibo',
 'Kabul Center',
 'Yerevan',
 'Baku',
 'Chittagong',
 'Phnom Penh',
 'Bat Dambang',
 'Tbilisi',
 'Agra',
 'Amritsar',
 'Aurangabad',
 'Bhopal',
 'Dhanbad',
 'Indore',
 'Kalyan Dombivali',
 'Kanpur',
 'Calcutta',
 'Lucknow',

In [29]:
non_financial_cities += balancers
non_financial_cities

['Chongqing',
 'Lagos',
 'Dhaka',
 'Cairo',
 'Karachi',
 'Kolkata',
 'Tehran',
 'Kinshasa',
 'Baoding',
 'Lahore',
 'Lima',
 'Bangalore',
 'Ho Chi Minh City',
 'Harbin',
 'Wuhan',
 'Shijiazhuang',
 'Bogotá',
 'Suzhou',
 'Linyi',
 'Chennai',
 'Nagoya',
 'Nanyang',
 'Zhengzhou',
 'Hyderabad',
 'Surabaya',
 "Xi'an",
 'Quanzhou',
 'Dongguan',
 'Bandung',
 'Hanoi',
 'Shenyang',
 'Baghdad',
 'Onitsha',
 'Ahmedabad',
 'Luanda',
 'Dallas',
 'Pune',
 'Houston',
 'Düsseldorf',
 'Surat',
 'Ouagadougou',
 'Abidjan',
 'Addis Ababa',
 'Kumasi',
 'Conakry',
 'Nairobi',
 'Antananarivo',
 'Fès',
 'Maputo',
 'Niamey',
 'Dakar',
 'Freetown',
 'Mogadishu',
 'Kampala',
 'Dar es Salaam',
 'Lusaka',
 'Harare',
 'Calgary',
 'Montréal',
 'Havana',
 'Santo Domingo',
 'Tegucigalpa',
 'Guadalajara',
 'Juárez',
 'La Laguna',
 'León',
 'Monterrey',
 'Puebla-Tlaxcala',
 'Querétaro',
 'San Luis Potosí-Soledad de Graciano Sánchez',
 'Tijuana',
 'Toluca',
 'Philadelphia',
 'Phoenix',
 'San Antonio',
 'San Diego',
 'Cór

In [30]:
len(non_financial_cities)

203

In [83]:
# pd.DataFrame(non_financial_cities).to_csv('data/cities_ratings_null.csv')

non_financial_cities = pd.read_csv('data/cities_ratings_null.csv', index_col=0)
non_financial_cities

,0
0,Chongqing
1,Lagos
2,Dhaka
3,Cairo
4,Karachi
...,...
198,Birmingham
199,Adelaide
200,Brisbaine
201,Perth


### Add non-financial cities to ratings df without rating

In [91]:
# Combine rated cities with non-rated cities
df = df.append([{'Centre': c, 'Rating': np.NaN} for c in non_financial_cities.iloc[:,0]], ignore_index=True)

# Remove duplicates
df.drop(index=df[df.Centre.duplicated()].index, inplace=True)
df.reset_index(inplace=True, drop=True)
df

,Centre,Rating
0,New York City,790.0
1,London,773.0
2,Hong Kong,771.0
3,Singapore,762.0
4,Shanghai,761.0
...,...,...
297,Birmingham,NaN
298,Adelaide,NaN
299,Brisbaine,NaN
300,Perth,NaN


In [92]:
# Prepare for merging with cities population
df['city'] = df.Centre.str.lower()
df

,Centre,Rating,city
0,New York City,790.0,new york city
1,London,773.0,london
2,Hong Kong,771.0,hong kong
3,Singapore,762.0,singapore
4,Shanghai,761.0,shanghai
...,...,...,...
297,Birmingham,NaN,birmingham
298,Adelaide,NaN,adelaide
299,Brisbaine,NaN,brisbaine
300,Perth,NaN,perth


In [57]:
cities_pop_regions = pd.read_csv('data/cities_pop_regions.csv', index_col=0)
cities_pop_regions

,city,region,country,population,geometry
0,tokyo,40,jp,31480498,"{'type': 'Point', 'coordinates': [139.751389, ..."
1,shanghai,23,cn,14608512,"{'type': 'Point', 'coordinates': [121.399722, ..."
2,bombay,16,in,12692717,"{'type': 'Point', 'coordinates': [72.825833, 1..."
3,karachi,05,pk,11627378,"{'type': 'Point', 'coordinates': [67.0822, 24...."
4,delhi,07,in,10928270,"{'type': 'Point', 'coordinates': [77.216667, 2..."
...,...,...,...,...,...
615,asansol,28,in,504320,"{'type': 'Point', 'coordinates': [86.983333, 2..."
616,arak,34,ir,503673,"{'type': 'Point', 'coordinates': [49.689164, 3..."
617,astrakhan,07,ru,502537,"{'type': 'Point', 'coordinates': [48.040756, 4..."
618,zhuhai,30,cn,501199,"{'type': 'Point', 'coordinates': [113.567778, ..."


In [93]:
cities_df = pd.merge(df, cities_pop_regions, how='left', on='city')
cities_df

,Centre,Rating,city,region,country,population,geometry
0,New York City,790.0,new york city,NaN,NaN,NaN,NaN
1,London,773.0,london,H9,gb,7421228.0,"{'type': 'Point', 'coordinates': [-0.093689, 5..."
2,Hong Kong,771.0,hong kong,NaN,NaN,NaN,NaN
3,Singapore,762.0,singapore,00,sg,3547809.0,"{'type': 'Point', 'coordinates': [103.8558333,..."
4,Shanghai,761.0,shanghai,23,cn,14608512.0,"{'type': 'Point', 'coordinates': [121.399722, ..."
...,...,...,...,...,...,...,...
300,Birmingham,NaN,birmingham,A7,gb,984336.0,"{'type': 'Point', 'coordinates': [-1.916667, 5..."
301,Adelaide,NaN,adelaide,05,au,1074168.0,"{'type': 'Point', 'coordinates': [138.598633, ..."
302,Brisbaine,NaN,brisbaine,NaN,NaN,NaN,NaN
303,Perth,NaN,perth,08,au,1446715.0,"{'type': 'Point', 'coordinates': [115.861397, ..."


In [97]:
cities_df.drop(index=cities_df[cities_df.city.duplicated() == True].index, inplace=True)
cities_df

,Centre,Rating,city,region,country,population,geometry
0,New York City,790.0,new york city,NaN,NaN,NaN,NaN
1,London,773.0,london,H9,gb,7421228.0,"{'type': 'Point', 'coordinates': [-0.093689, 5..."
2,Hong Kong,771.0,hong kong,NaN,NaN,NaN,NaN
3,Singapore,762.0,singapore,00,sg,3547809.0,"{'type': 'Point', 'coordinates': [103.8558333,..."
4,Shanghai,761.0,shanghai,23,cn,14608512.0,"{'type': 'Point', 'coordinates': [121.399722, ..."
...,...,...,...,...,...,...,...
300,Birmingham,NaN,birmingham,A7,gb,984336.0,"{'type': 'Point', 'coordinates': [-1.916667, 5..."
301,Adelaide,NaN,adelaide,05,au,1074168.0,"{'type': 'Point', 'coordinates': [138.598633, ..."
302,Brisbaine,NaN,brisbaine,NaN,NaN,NaN,NaN
303,Perth,NaN,perth,08,au,1446715.0,"{'type': 'Point', 'coordinates': [115.861397, ..."


In [98]:
cities_df.geometry.value_counts(dropna=False)

NaN                                                           116
{'type': 'Point', 'coordinates': [19.083333, 47.5]}             1
{'type': 'Point', 'coordinates': [-3.69222, 40.408566]}         1
{'type': 'Point', 'coordinates': [-9.133333, 38.716667]}        1
{'type': 'Point', 'coordinates': [106.552778, 29.562778]}       1
                                                             ... 
{'type': 'Point', 'coordinates': [67.0822, 24.9056]}            1
{'type': 'Point', 'coordinates': [12.583333, 55.666667]}        1
{'type': 'Point', 'coordinates': [32.484636, 37.871354]}        1
{'type': 'Point', 'coordinates': [-60.639317, -32.946818]}      1
{'type': 'Point', 'coordinates': [44.3938889, 33.3386111]}      1
Name: geometry, Length: 187, dtype: int64

In [71]:
cities_df.city.duplicated?

## Generate city grids from geojson boundaries

In [2]:
def get_nominatim_search_url(city_name):
    """
    Encodes city name and wraps it in openstreetmap request for geospatial city boundaries
    """
    if len(city_name) <= 0:
        return None
    return f"https://nominatim.openstreetmap.org/search.php?q={quote_plus(city_name)}&polygon_geojson=1&format=json"

In [3]:
def bounds_from_nominatim(geocode_json):
    for g in geocode_json:
        geo_json = g.get('geojson')

        if geo_json and geo_json['type'] in ('Polygon', 'MultiPolygon'):
            return g
    return None

def generate_and_save_city_grid(c, grid_threshold=0.05):
    url = get_nominatim_search_url(c)
    print('url', url)
    
    geocode_data = requests.get(url)
    geocode_json = geocode_data.json()
    
    city_filename = re.sub(r'[^A-Za-z]','',c)

    if len(geocode_json):
        geocode_data = bounds_from_nominatim(geocode_json)
        
        if not geocode_data:
            print('Unable to Find City')
            return
        
        lat = geocode_data['lat']
        lng = geocode_data['lon']

        with open(f'data/{city_filename}.geojson', 'w') as f:
           json.dump(geocode_data['geojson'], f)
        
        geo_df = gpd.read_file(f'data/{city_filename}.geojson')

        grid = generate_square_grid(geo_df.iloc[0]['geometry'], grid_threshold)

        ## apply to feature and feature collection?
        cfs = {
            # TODO
            'city_name': c,
        }

        feature_list = [Feature(geometry=g) for g in grid]
        feature_collection = FeatureCollection(feature_list) 
        
        with open(f'data/{city_filename}_grid.geojson', 'w') as f:
           dump(feature_collection, f)
        
        print('ready to plot', lat, lng, len(grid))
        
        return plot_foursquare_search_grid(lat, lng, grid)

In [ ]:
def generate_city_grid(c, gdf, grid_threshold=0.05):
    
    city_filename = re.sub(r'[^A-Za-z]','',c)

#         with open(f'data/{city_filename}.geojson', 'w') as f:
#            json.dump(geocode_data['geojson'], f)
        
#         geo_df = gpd.read_file(f'data/{city_filename}.geojson')

    grid = generate_square_grid(gdf.geometry, grid_threshold)

    cfs = {'city_name': c}

    feature_list = [Feature(geometry=g) for g in grid]
    feature_collection = FeatureCollection(feature_list) 

    with open(f'data/{city_filename}_districts_grid.geojson', 'w') as f:
       dump(feature_collection, f)

    print('Saved grid to file.')

In [6]:
[generate_and_save_city_grid(i, 0.05) for i in df.Centre]

url https://nominatim.openstreetmap.org/search.php?q=S%C3%A3o+Paulo&polygon_geojson=1&format=json
0.4612008000000003
0.6501452000000008
grid length 87
ready to plot -23.5506507 -46.6333824 87
url https://nominatim.openstreetmap.org/search.php?q=Cyprus&polygon_geojson=1&format=json
2.832560100000002
1.474881400000001
next threshold 0.1
grid length 281
ready to plot 34.9823018 33.1451285 281
url https://nominatim.openstreetmap.org/search.php?q=Sofia&polygon_geojson=1&format=json
0.23167250000000195
0.1667065999999977
grid length 19
ready to plot 42.6978634 23.3221789 19
url https://nominatim.openstreetmap.org/search.php?q=Riga&polygon_geojson=1&format=json
0.39247310000000013
0.22863530000000054
grid length 35
ready to plot 56.9493977 24.1051846 35
url https://nominatim.openstreetmap.org/search.php?q=Panama&polygon_geojson=1&format=json
5.893149400000013
2.836307800000001
next threshold 0.2
grid length 328
ready to plot 8.559559 -81.1308434 328
url https://nominatim.openstreetmap.org/sea

In [36]:
[generate_and_save_city_grid(i, 0.05) for i in non_financial_cities.iloc[40:,0]]

url https://nominatim.openstreetmap.org/search.php?q=Ouagadougou&polygon_geojson=1&format=json
0.24668749999999995
0.23992469999999955
grid length 28
ready to plot 12.3704741 -1.5334099725592838 28
url https://nominatim.openstreetmap.org/search.php?q=Abidjan&polygon_geojson=1&format=json
0.5763377000000003
0.41545589999999955
grid length 91
ready to plot 5.4091179 -4.0422099 91
url https://nominatim.openstreetmap.org/search.php?q=Addis+Ababa&polygon_geojson=1&format=json
0.2545999999999964
0.2663808000000003
grid length 28
ready to plot 9 38.75 28
url https://nominatim.openstreetmap.org/search.php?q=Kumasi&polygon_geojson=1&format=json
0.0003641999999999257
0.00035929999999950724
grid length 1
ready to plot 49.192787100000004 -2.123797886441899 1
url https://nominatim.openstreetmap.org/search.php?q=Conakry&polygon_geojson=1&format=json
0.3664709999999989
0.3211756000000001
grid length 32
ready to plot 9.6216542 -13.559258646382819 32
url https://nominatim.openstreetmap.org/search.php?q

url https://nominatim.openstreetmap.org/search.php?q=Guarulhos&polygon_geojson=1&format=json
0.2351109999999963
0.2363509999999991
grid length 23
ready to plot -23.4430602 -46.524459 23
url https://nominatim.openstreetmap.org/search.php?q=Manaus&polygon_geojson=1&format=json
1.6410000000000053
1.298
grid length 435
ready to plot -3.1316333 -59.9825041 435
url https://nominatim.openstreetmap.org/search.php?q=Salvador&polygon_geojson=1&format=json
2.5439580999999976
1.4750028000000004
next threshold 0.1
grid length 260
ready to plot 13.8000382 -88.9140683 260
url https://nominatim.openstreetmap.org/search.php?q=Barranquilla&polygon_geojson=1&format=json
0.16438970000000097
0.1914415999999992
grid length 14
ready to plot 11.011848449999999 -74.80886011839334 14
url https://nominatim.openstreetmap.org/search.php?q=Cali&polygon_geojson=1&format=json
0.24804589999999394
0.2751684999999999
grid length 30
ready to plot 3.4108435 -76.58121271109312 30
url https://nominatim.openstreetmap.org/sea

0.3878240999999889
0.3616228999999995
grid length 56
ready to plot -6.1734808 106.4992042 56
url https://nominatim.openstreetmap.org/search.php?q=Ahwaz&polygon_geojson=1&format=json
0.31442159999999575
0.4079025000000023
grid length 40
ready to plot 31.32628875 48.74431395696944 40
url https://nominatim.openstreetmap.org/search.php?q=Esfahan&polygon_geojson=1&format=json
5.860543399999997
3.8100105000000006
next threshold 0.2
grid length 320
ready to plot 33.196733 52.504026 320
url https://nominatim.openstreetmap.org/search.php?q=Karaj&polygon_geojson=1&format=json
0.29005569999999636
0.13308500000000123
grid length 16
ready to plot 35.8228964 50.95876430889608 16
url https://nominatim.openstreetmap.org/search.php?q=Mashhad&polygon_geojson=1&format=json
0.3041680999999983
0.3143703000000002
grid length 32
ready to plot 36.29275685 59.57965157590017 32
url https://nominatim.openstreetmap.org/search.php?q=Qom&polygon_geojson=1&format=json
0.16551880000000097
0.1535306000000034
grid leng

3.1779607000000034
2.522838700000001
next threshold 0.1
grid length 501
ready to plot 38.0211951 32.5224943 501
url https://nominatim.openstreetmap.org/search.php?q=Manisa&polygon_geojson=1&format=json
2.0178152999999988
1.3218154999999996
next threshold 0.1
grid length 173
ready to plot 38.8574402 28.0565711 173
url https://nominatim.openstreetmap.org/search.php?q=Mersin&polygon_geojson=1&format=json
2.7565490999999938
1.618600899999997
next threshold 0.1
grid length 272
ready to plot 36.8328277 33.9685895 272
url https://nominatim.openstreetmap.org/search.php?q=Sanliurfa&polygon_geojson=1&format=json
2.403280600000002
1.3514142999999947
next threshold 0.1
grid length 238
ready to plot 37.2595198 39.0408174 238
url https://nominatim.openstreetmap.org/search.php?q=Tashkent&polygon_geojson=1&format=json
0.351638400000013
0.25339479999999526
grid length 29
ready to plot 41.283974549999996 69.25598969313666 29
url https://nominatim.openstreetmap.org/search.php?q=Sana%27a&polygon_geojson=1

 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [5]:
missing_cities = ['Accra',
 'Alexandria',
 'Algiers',
 'Ankara',
 'Antananarivo',
 'Ashgabat',
 'Asuncion Metro',
 'Atlanta',
 'Bamako',
 'Bangui',
 'Beirut',
 'Benghazi',
 'Bergen',
 'Bishkek',
 'Bratislava',
 'Brazzaville',
 'Brisbane',
 'Cameroon',
 'Cape Town',
 'Caracas',
 'Chisinau',
 'Christchurch',
 'Colombo',
 'Cork',
 'Cotonou',
 'Delhi',
 'Denmark',
 'Douala',
 'Dushanbe',
 'Gaborone',
 'Gaza',
 'Goteborg',
 'Guatemala City',
 'Guayaquil',
 'Israel',
 'Jeddah',
 'Kaunas',
 'Khartoum',
 'Kiev',
 'Kigali',
 'Kinshasa',
 'Kumasi',
 'Kuwait City',
 'La Paz',
 'Lahore',
 'Lefkosia',
 'Libreville',
 'Limassol',
 'Ljubljana',
 'Lome',
 'Male',
 'Managua',
 'Manama',
 'Maseru',
 'Mecca',
 'Medina',
 'Miami',
 'Monrovia',
 'NDjamena',
 'Nicosia',
 'Nouakchott',
 'Pafos',
 'Pointe-Noire',
 'Port-Louis',
 'Port-au-Prince',
 'Prishtine',
 'San Jose',
 'San Miguelito',
 'San Pedro Sula',
 'San Salvador',
 'Sanaa',
 'Sarajevo',
 'Skopje',
 'South Sudan',
 'Thessaloniki',
 'Tirana',
 'Tripoli',
 'Tunis',
 'Ulaanbaatar',
 'Vientiane/Vianchan',
 'Vilnius',
 'Wellington',
 'Windhoek']

In [6]:
[generate_and_save_city_grid(i, 0.05) for i in missing_cities]

url https://nominatim.openstreetmap.org/search.php?q=Accra&polygon_geojson=1&format=json
0.00013700000000005375
8.640000000070813e-05
grid length 1
ready to plot 51.4503833 -0.9079996000000001 1
url https://nominatim.openstreetmap.org/search.php?q=Alexandria&polygon_geojson=1&format=json
0.10731979999999908
0.06052629999999937
grid length 6
ready to plot 38.8147596 -77.09024765272724 6
url https://nominatim.openstreetmap.org/search.php?q=Algiers&polygon_geojson=1&format=json
20.666245
18.3280585
next threshold 0.2
grid length 5608
ready to plot 28.0000272 2.9999825 5608
url https://nominatim.openstreetmap.org/search.php?q=Ankara&polygon_geojson=1&format=json
3.039952099999997
2.0964607000000015
next threshold 0.1
grid length 340
ready to plot 39.7160439 32.7059948 340
url https://nominatim.openstreetmap.org/search.php?q=Antananarivo&polygon_geojson=1&format=json
Unable to Find City
url https://nominatim.openstreetmap.org/search.php?q=Ashgabat&polygon_geojson=1&format=json
0.48636650000

0.07042390000000154
0.05836610000000064
grid length 5
ready to plot 35.18089155 33.37325855497367 5
url https://nominatim.openstreetmap.org/search.php?q=Libreville&polygon_geojson=1&format=json
0.8701606999999996
0.5987337
grid length 156
ready to plot 0.450202 9.516160556041065 156
url https://nominatim.openstreetmap.org/search.php?q=Limassol&polygon_geojson=1&format=json
0.6413322000000008
0.3307889000000017
grid length 73
ready to plot 34.79489125 32.985589384507975 73
url https://nominatim.openstreetmap.org/search.php?q=Ljubljana&polygon_geojson=1&format=json
0.34671499999999966
0.17193309999999684
grid length 25
ready to plot 46.0499803 14.5068602 25
url https://nominatim.openstreetmap.org/search.php?q=Lome&polygon_geojson=1&format=json
0.13029539999999984
0.09174620000000022
grid length 10
ready to plot 6.15803455 1.2516823732466333 10
url https://nominatim.openstreetmap.org/search.php?q=Male&polygon_geojson=1&format=json
16.5076663
14.853272999999998
next threshold 0.2
grid leng

 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [4]:
missing_cities = [
 'Algiers',
 'Cameroon',
 'South Sudan']

In [5]:
[generate_and_save_city_grid(i, 0.05) for i in missing_cities]

url https://nominatim.openstreetmap.org/search.php?q=Algiers&polygon_geojson=1&format=json
20.666245
18.3280585
next threshold 0.5
next threshold 2.0
grid length 75
ready to plot 28.0000272 2.9999825 75
url https://nominatim.openstreetmap.org/search.php?q=Cameroon&polygon_geojson=1&format=json
7.806379700000001
11.434351
next threshold 0.5
next threshold 2.0
grid length 20
ready to plot 4.6125522 13.1535811 20
url https://nominatim.openstreetmap.org/search.php?q=South+Sudan&polygon_geojson=1&format=json
12.501218999999999
8.747409000000001
next threshold 0.5
next threshold 2.0
grid length 26
ready to plot 7.8699431 29.6667897 26
